In [1]:
# import packages
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
import time 
import os
import json
import urllib


In [2]:
# Some urls are blocked from untrusted sites, the below code make python environment to trust every site
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
getattr(ssl, '_create_unverified_context', None)):
    ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
def download_images():
    
    f = open('site_urls.txt','r')
    search_url = f.readlines()[18]
    print(search_url)
   
    # path of chorme drive to run selenium
    path = r'chromedriver'
    
    driver = webdriver.Chrome(path)
    
    k = open('wildlife keywords.txt','r')
    kw = k.readlines()
    
    for i in range(0,len(kw)):
        print(i)
        keyW = kw[i]
        
        driver.get(search_url)

        time.sleep(2)
        
        # search text box
        box = driver.find_element_by_xpath('//*[@id="header-search-left"]/input[1]')  
        box.clear()
        
        #sending the keyword
        box.send_keys(keyW)
        time.sleep(2)
        
        # try block if there are no ad results for the keyword it will continue with next keyword
        try:
            # collecting links of ad results
            links = driver.find_element_by_xpath('//*[@id="items-icon"]')
            sub_lnks = links.find_elements_by_tag_name('a')

            lnks_lst = []

            for lnk in sub_lnks:
                lnks_lst.append(lnk.get_attribute('href'))
            print(lnks_lst[0:3])

            # making a directory like URL+Keyword_number , example 18_0 , 18_1,...
            if lnks_lst:
                if not os.path.exists(str(18)+'_'+str(i)):
                    os.mkdir(str(18)+'_'+str(i))
            
            #  iterating through ads results 
            for j in range(0,2):
                driver.get(lnks_lst[j])
                print(lnks_lst[j])
                time.sleep(2)
                # Title capturing 
                sub_txt = driver.find_element_by_xpath('//*[@id="item-title"]/h1').text
                #making directory inside the above created directory like 0 means first ad, 1 for second ad....
                if not os.path.exists(str(18)+'_'+str(i) +'/'+ str(j)):
                    os.mkdir(str(18)+'_'+str(i) +'/'+ str(j))
                # making dictionary
                details = {
                    'name' : sub_txt,
                    'price' : driver.find_element_by_xpath('//*[@id="item-price-digital"]').text,
                    'location': driver.find_element_by_xpath('//*[@id="item-seller"]').get_attribute("innerText"),
                    'details': None,
                    'description' : None,
                    'url' : lnks_lst[j]
                }
                # writing the dictionary to text file like 0.txt for first ad ....
                with open(str(18)+'_'+str(i) +'/'+ str(j) + '/' + str(j) + '.txt', 'w') as tfile:
                     tfile.write(json.dumps(details))
                
                # collecting image urls
                image_lst = []
                elem1 = driver.find_element_by_xpath('//*[@id="item-picture-additional"]/div')
                sub = elem1.find_elements_by_tag_name('img')
                for k,l in enumerate(sub):
                    src = l.get_attribute('src')
                    src = src.replace('thumb','')
                    image_lst.append(src)
                    print(src)
                    # collecting images and saving it inside the above created directory like 0_0.jpg for first ad first image
                    # 0_1.jpg for first ad second image ...
                    if src != None:
                        urllib.request.urlretrieve(str(src), os.path.join(str(18)+'_'+str(i), str(j), str(j)+'_'+str(k)+'.jpg'))
                # writing image urls to text file like 0_imgs.txt for first ad ....
                with open(str(18)+'_'+str(i) +'/'+ str(j) + '/' + str(j) + '_imgs.txt', 'w') as tfile:
                     tfile.write(json.dumps(image_lst)) 
        except:
            continue
            
    driver.close()
    

In [4]:
def main():
    download_images()
    
if __name__ == "__main__":
    main()

http://www.trocadero.com/directory/search.php?keyword=[KW]&archinsearch=yes&click_to_search_now_x=0&click_to_search_now_y=0&category=ALL%20ITEMS&period=No%20Selection&sortfield=calculatedPrice&sortdir=SORT_DESC&start=0&l=60&view=icon

0
['https://www.trocadero.com/stores/lenfantgallery/items/1464015/Pair-Faux-Alligator-Waste-Paper-Baskets-11-10', 'https://www.trocadero.com/stores/lenfantgallery/', 'https://www.trocadero.com/stores/lenfantgallery/items/1464015/Pair-Faux-Alligator-Waste-Paper-Baskets-11-10']
https://www.trocadero.com/stores/lenfantgallery/items/1464015/Pair-Faux-Alligator-Waste-Paper-Baskets-11-10
https://images.trocadero.com/stores/lenfantgallery/items/1464015/picture1.jpg
https://www.trocadero.com/stores/lenfantgallery/
1
